In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import wave
import math


#Function to define the positions of the bits to change


In [3]:
def sequence_generator(byte_array_length,key,message_length):
  
  bits = []
  n = byte_array_length
  key = int(key)

  for i in range(message_length):
    position = (15485863*key + 2038074743*i)%n
    while(position in range(0,33)):
      position = (15485863*key + 2038074743*i)%n
      
    bits.append(position)

  return bits

#Function to encode the message in the audio

In [4]:
def encode(byte_array,key,message,audio_output_path):
    # Convert the text data to a bit array
    message = [int(i) for i in ''.join([bin(ord(c)).lstrip('0b').rjust(8,'0') for c in message])]

    message_len = len(message)

    message_len_bin = bin(message_len)[2:].zfill(32)
    
    message_len_bin = list(message_len_bin)
    
    for i in range(0,32):
      byte_array[i] = ((byte_array[i] & 254) | int(message_len_bin[i]))

    positions = sequence_generator(len(byte_array),key,message_len)
    
    # Iterate over the text bits and hide them in the LSB of the audio data
    for i in range(len(positions)):
        byte_array[positions[i]] = ((byte_array[positions[i]] & 254) | message[i])

    return byte_array

In [5]:
def run_encode(audio_path,key,message,audio_output_path):
  # Open the audio file in read mode
  with wave.open(audio_path, "rb") as audio_file:
    # Read the frames of the audio file
    frames = audio_file.readframes(-1)
    # Convert the frames to a byte array
    byte_array = bytearray(frames)
    byte_array = encode(byte_array,key,message,audio_output_path)
  
  # Write the modified audio data to a new file
    with wave.open(audio_output_path, "wb") as hidden_audio:
        hidden_audio.setparams(audio_file.getparams())
        hidden_audio.writeframes(byte_array)
  
  return "The data has been hidden successfully"

#Function to decode the audio

In [6]:
def run_decode(audio_output_path,key):
  with wave.open(audio_output_path, "rb") as audio_file:
    # Read the frames of the audio file
    frames = audio_file.readframes(-1)
    # Convert the frames to a byte array
    byte_array = bytearray(frames)

  message_len_bin = ""
  for i in range(0,32):
    bit = bin(byte_array[i])[2:].zfill(8)
    message_len_bin += str(bit[-1])

  message_len = int(message_len_bin,2)

  positions = sequence_generator(len(byte_array),key,message_len)
  
  message = []

  for i in range(len(positions)):
    message.append(byte_array[positions[i]] & 1)
  
  text = ''.join([chr(int(i, 2)) for i in [''.join(map(str,message[i:i+8])) for i in range(0, len(message), 8)]])

  return text


#Run the program

In [8]:
audio_path = input("Enter audio path:")
key = input("Enter key:")
message = input("Enter message:")

audio_output_path = input("Enter output path for audio:")

print(run_encode(audio_path,key,message,audio_output_path))
print("The decoded text from the audio is:",run_decode(audio_output_path,key))

Enter audio path:/content/cover_audio.wav
Enter key:4321
Enter message:hekko
Enter output path for audio:/content/cover_audio_out.wav
The data has been hidden successfully
The decoded text from the audio is: hekko
